<a href="https://colab.research.google.com/github/chanphoomk/PythonConcurrentWebScraping_1/blob/master/Parallel_Trial_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Summary ***

After Scraping for on website, with 2 second time stop, the results are shown below.

**1 page, 10 Articles and links**
* Normal Scraping took around 2.59 second, 
* Concurrent Scraping took around 2.01 second, 
* faster than 1.29 times
 

**10 pages, 100 Articles and links**
* Normal Scraping took around 25.33 second,
* Concurrent Scraping took around 2.073 second,
* faster than 12.22 times

**50 pages, 500 Articles and links**
* Normal Scraping took around 135.07 second,
* Concurrent Scraping took around 3.58 second,
* faster than 37.73 times


In [0]:
import requests 
import time
from bs4 import BeautifulSoup as soup

def scrape_page(num_page):
  #Way to fix missing data in the tag (TRY, EXECPT)
  #print(num_page)
  result=requests.get(f"https://thestandard.co/category/news/world/page/{num_page}")
  src = result.content
  #To be able to pass HTML content
  soup = BeautifulSoup(src,'lxml')

  for article in soup.find_all("h3",class_='news-title'):
    headline = article.a.text
    headline = headline.replace('\n', '') #Remove new line
    news_src = article.find('a')['href']

    post_content.append(headline)
    link_content.append(news_src)
    view_content.append(find_total_view(news_src))

  scrape_date(num_page)
    
def scrape_date(num_page):
  result=requests.get(f"https://thestandard.co/category/news/world/page/{num_page}")
  src = result.content
  #To be able to pass HTML content
  soup = BeautifulSoup(src,'lxml')
  for date_collect in soup.find_all("div",class_='date'):
    date = date_collect.text
    date = date.replace('\n', '')
    date_content.append(date)

def total_page():
  result=requests.get(f"https://thestandard.co/category/news/world")
  src = result.content
  #To be able to pass HTML content
  soup = BeautifulSoup(src,'lxml')
  tt_page = soup.find("span",class_='pages').text
  tt_page = int(tt_page.replace('1 of ', ''))
  return tt_page

In [0]:
# install chromium, its driver, and selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# open it, go to a website, and get results
#wd = webdriver.Chrome('chromedriver',options=options)
#wd.get("https://www.website.com")
#print(wd.page_source)  # results

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [92.5 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:14 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 

In [0]:
def find_total_view(link):
    driver = webdriver.Chrome('chromedriver',options=options)
    driver.get("https://thestandard.co/moon-jae-in-party-won-the-election-in-south-korea/")
    time.sleep(3)

    page_html = driver.page_source
    #driver.close()
    data = soup(page_html,'html.parser') #scan data
    span = data.findAll('span', {'class' : 'entry-view'})
    span = span[0].findAll('span')

    tt_view = span[0].text

    return tt_view

def scrape_date_from_page(web_link):
  result=requests.get(web_link)
  src = result.content
  soup = BeautifulSoup(src,'lxml')
  raw_date = soup.find("div", {'class' : "meta-date"})
  date = raw_date.find('span').text
  return date


In [0]:
find_total_view('https://thestandard.co/germany-slowly-eases-lockdown/')

'979'



```
# This is formatted as code
```

# ***Normal Scraping***

In [0]:
post_content = []
link_content = []

start = time.perf_counter()

page_to_scrape=50

for num_page in range(1,page_to_scrape+1):
    scrape_page(num_page)
    time.sleep(2)
end = time.perf_counter()

print(f"time use for normal scrape: {round(end-start  ,3)}")

NameError: ignored

In [0]:
import concurrent.futures

#Run for heading and link
start = time.perf_counter()

post_content = []
link_content = []
date_content = []
view_content = []

with concurrent.futures.ThreadPoolExecutor() as executor:

  # page_to_scrape = int(total_page())
  page_to_scrape = 1
  
  pages=[]
  for i in range(1,page_to_scrape+1):
    pages.append(i)

  #pages=[1,2,3,4,5,6,7,8,9,10]
  executor.map(scrape_page,pages)
  time.sleep(2)
    
end = time.perf_counter()

print(f"time use: {round(end-start,3)}")

time use: 2.008


# ***Data to CSV***

In [0]:
#Insert Data to CSV file

# importing pandas as pd  
import pandas as pd  
   
# dictionary of lists  
dict = {'date':date_content, 'header': post_content, 'link': link_content}  
     
df = pd.DataFrame(dict) 
  
# saving the dataframe 
df.to_csv(r'/content/data.csv', index=False) 